In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from textblob import TextBlob
import string
import re

In [2]:
data = pd.read_csv('alteryx_community.csv')

In [3]:
data.loc[data.reply_time=='No replies','reply_time'] = np.NAN
data['reply_time'] = pd.to_datetime(data['reply_time'])
data['post_time'] = pd.to_datetime(data['post_time'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   post_author   83134 non-null  object        
 1   post_message  83439 non-null  object        
 2   post_time     83439 non-null  datetime64[ns]
 3   reply_author  83436 non-null  object        
 4   reply_time    61434 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 3.2+ MB


In [4]:
data.to_csv('../Shiny/clean_data.csv', index = False)

In [5]:
#all the string to lower cases
data['post_message1'] = data['post_message'].str.lower()
# \S+ means anything that is not an empty space
data['post_message1'] = data['post_message1'].apply(lambda x: re.sub('http\S*', '', x))
# \s+ means all empty space (\n, \r, \t)
data['post_message1'] = data['post_message1'].apply(lambda x: re.sub('\s+', ' ', x))
data['post_message1'] = data['post_message1'].apply(lambda x: re.sub('[^\w\s]', '', x))

In [6]:
data.sample(10)

,post_author,post_message,post_time,reply_author,reply_time,post_message1
61914,asmagar,Re: Tool Mastery | Python,2019-05-22,SydneyF,2019-05-22,re tool mastery python
12260,AlexStewart628,Moving rows into column for multiple files,2018-01-30,No replies,NaT,moving rows into column for multiple files
43881,jagjit_singh,YTD Calculations,2015-10-28,joe_strellis,2015-10-29,ytd calculations
17648,justinpopat,Replace if New Rows in Oracle,2018-07-24,justinpopat,2018-07-25,replace if new rows in oracle
78062,AdamSpencer,Education Use Case?,2021-04-28,AdamSpencer,2021-04-28,education use case
53518,_arun_gurubaram,Downloading data with spatial object from SDE ...,2012-07-24,zendonadi,2018-09-25,downloading data with spatial object from sde ...
68300,PaulFields2,Need to create an empty table in oracle,2020-12-22,PaulFields2,2020-12-23,need to create an empty table in oracle
77850,ISUGraber,Convert Text to 24 Hour Time,2021-05-01,No replies,NaT,convert text to 24 hour time
3162,bsharbo,Pass Paramater through Chained Applications,2016-06-01,No replies,NaT,pass paramater through chained applications
14805,rhyatt,Order number field within a group,2017-06-19,No replies,NaT,order number field within a group


In [7]:
best_rev_corpus = ' '.join(data['post_message1'])

In [8]:
lemmatizer= WordNetLemmatizer()

In [9]:
stop = stopwords.words('english') + list(string.punctuation)
data['post_message1'] = data['post_message1'].apply(lambda text: " ".join(word for word in text.split() if word not in stop))

In [10]:
best_rev_tokens = [lemmatizer.lemmatize(tokens) for tokens in word_tokenize(best_rev_corpus) if tokens not in stop]

fdist = FreqDist(best_rev_tokens)
fdist.most_common(50)

[('data', 10662),
 ('tool', 10374),
 ('file', 9991),
 ('alteryx', 7942),
 ('column', 7033),
 ('error', 6645),
 ('excel', 6020),
 ('input', 5805),
 ('output', 5780),
 ('multiple', 5386),
 ('row', 4626),
 ('date', 4323),
 ('field', 4248),
 ('value', 4066),
 ('using', 4036),
 ('workflow', 3918),
 ('macro', 3602),
 ('table', 3324),
 ('formula', 3086),
 ('dynamic', 2592),
 ('help', 2451),
 ('name', 2302),
 ('based', 2295),
 ('one', 2173),
 ('sheet', 2134),
 ('number', 2081),
 ('string', 2046),
 ('text', 2039),
 ('sql', 1893),
 ('different', 1890),
 ('filter', 1791),
 ('record', 1780),
 ('issue', 1754),
 ('run', 1547),
 ('server', 1504),
 ('time', 1503),
 ('format', 1492),
 ('join', 1476),
 ('tab', 1466),
 ('new', 1457),
 ('list', 1367),
 ('app', 1359),
 ('create', 1330),
 ('email', 1324),
 ('specific', 1314),
 ('two', 1266),
 ('download', 1213),
 ('connection', 1183),
 ('regex', 1129),
 ('designer', 1127)]